In [21]:
from pyelasticsearch import ElasticSearch
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, MetaData, Table, select
from time import time

servername = "spmyfap00003"
database = "eb"
serverdriver = "sql server"

engine = create_engine('mssql+pyodbc://'+servername+'/'+database+'?driver='+serverdriver)

In [88]:
chunk = pd.read_sql('select claimnumber,patientname,companyname,diagnosisdescription,incurred \
                    from rclmext where datevisit = 20160104',engine, chunksize = 100)

In [81]:
chunk

<generator object _query_iterator at 0x000000000C91C4C8>

In [23]:
es = ElasticSearch('http://localhost:9200/')

In [85]:
settings ={
    'test':{
    'properties':{
    'companyname':{'index':'not_analyzed','type': 'string'},
    'diagnosisdescription':{'index':'not_analyzed', 'type':'string'}
         }
    }
}

In [86]:
try :
    es.delete_index("test")
except :
    pass

es.create_index("test", settings={'mappings': settings})

{u'acknowledged': True}

In [89]:
t0 = time()

print "now indexing %s..."%(database)

for i,df in enumerate(chunk): 
    print i
    records=df.where(pd.notnull(df), None).T.to_dict()
    list_records=[records[it] for it in records]
    try :
        es.bulk_index("test","test",list_records)
    except :
        print "error!, skiping some, sorry"
        pass

print "done in %.3fs"%(time()-t0)

now indexing eb...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
done in 6.745s
